In [1]:
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Dense, Conv2D,MaxPooling2D,Dropout,Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold,cross_val_score,train_test_split
import matplotlib.image as Image
import PIL.Image as Image


Using TensorFlow backend.


In [2]:
m_check = ModelCheckpoint("model/--{epoch:02d}--{val_loss:.4f}.hdf5", monitor = 'val_loss',save_best_only=True)


In [6]:
x = np.load("../train.npy")
print(x[0])

y = np.load("y.npy")
predx = Image.open("../495.png").resize(((150,112)))
predx= np.asarray(predx).reshape(1,112,150,3)

y = np_utils.to_categorical(y)

[[[1.         1.         1.        ]
  [0.99607843 0.99607843 0.99607843]
  [0.99607843 0.99607843 0.99607843]
  ...
  [0.99607843 0.99607843 0.99607843]
  [0.99607843 0.99607843 0.99607843]
  [1.         1.         1.        ]]

 [[1.         1.         1.        ]
  [0.36470588 0.36470588 0.36470588]
  [0.2        0.2        0.2       ]
  ...
  [0.2        0.2        0.2       ]
  [0.36470588 0.36470588 0.36470588]
  [1.         1.         1.        ]]

 [[1.         1.         1.        ]
  [0.18431373 0.18431373 0.18431373]
  [0.         0.         0.        ]
  ...
  [0.         0.         0.        ]
  [0.18431373 0.18431373 0.18431373]
  [1.         1.         1.        ]]

 ...

 [[1.         1.         1.        ]
  [0.18431373 0.18431373 0.18431373]
  [0.         0.         0.        ]
  ...
  [0.         0.         0.        ]
  [0.18431373 0.18431373 0.18431373]
  [1.         1.         1.        ]]

 [[1.         1.         1.        ]
  [0.36470588 0.36470588 0.36470588]


In [7]:
def create_model():
    model = Sequential()
    model.add(Conv2D(200,(3,3),padding='same',activation='relu',input_shape=(112,150,3)))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(200,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(2,2),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(2,2),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))

    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(9,activation="softmax"))

    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

    return model


In [8]:
seed = np.random.seed(7)

kf = KFold(n_splits=3, shuffle=True,random_state=seed)


In [9]:

x_train,x_test , y_train,y_test = train_test_split(x,y,shuffle=True , random_state=seed)

In [10]:
model = create_model()
for train_i,test_i in kf.split(x):
    train_x,train_y = x[train_i],y[train_i]
    test_x, test_y = x[test_i], y[test_i]

    model.fit(train_x,train_y,batch_size=30,epochs=100,validation_split=0.25,callbacks=[m_check])
    score = model.evaluate(test_x,test_y)
    print(score)

Train on 2250 samples, validate on 750 samples
Epoch 1/100
  60/2250 [..............................] - ETA: 6:21 - loss: 2.2455 - acc: 0.1667

KeyboardInterrupt: 

In [ ]:
predy = model.evaluate(x_test,y_test)
print(predy)